# Petite implémentation d'un RNN tout simple pour tester les fonctions d'importation de la Base de données faites par Tristan

In [1]:
import torch
from random import randint
from utilities import chordUtil
from utilities import dataImport
from utilities.chordUtil import *
from utilities.dataImport import *
from sklearn.model_selection import train_test_split
import os
import numpy as np
import random
import sys
import torch.nn as nn
import time
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [2]:
def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [3]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
print("use_cuda")
print(use_cuda)
device = torch.device("cuda:0" if use_cuda else "cpu")

use_cuda
True


In [4]:
# Init
lenSeq = 16
alpha = 'a1'
rootname = "inputs/jazz_xlab/"
filenames = os.listdir(rootname)
#filenames.remove(".DS_Store")
dictChord, listChord = chordUtil.getDictChord(eval(alpha))
print(len(dictChord))
print(dictChord)

37
{'G#:maj': 0, 'A#:maj': 1, 'A:min': 2, 'D:maj': 3, 'F#:dim': 4, 'C#:dim': 5, 'B:maj': 6, 'F:min': 7, 'E:min': 8, 'B:min': 9, 'C:min': 10, 'F:dim': 11, 'A#:min': 12, 'D#:maj': 13, 'D:dim': 14, 'G#:min': 15, 'C#:maj': 16, 'A:dim': 17, 'F:maj': 18, 'A#:dim': 19, 'F#:min': 20, 'G:maj': 21, 'D#:dim': 22, 'N': 23, 'C:dim': 24, 'A:maj': 25, 'E:dim': 26, 'G:min': 27, 'F#:maj': 28, 'E:maj': 29, 'B:dim': 30, 'D#:min': 31, 'G:dim': 32, 'D:min': 33, 'C:maj': 34, 'G#:dim': 35, 'C#:min': 36}


In [5]:
# Create datasets
files_train ,files_test = train_test_split(filenames,test_size=0.7)
dataset_train = dataImport.ChordSeqDataset(files_train, rootname, alpha, dictChord, lenSeq)
dataset_test = dataImport.ChordSeqDataset(files_test, rootname, alpha, dictChord, lenSeq)


In [6]:
# Create generators
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 6}
training_generator = data.DataLoader(dataset_train, **params)
testing_generator = data.DataLoader(dataset_test, **params)


In [7]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=0)

    def forward(self, input, hidden):
        input_combined = torch.cat((input, hidden), 0)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden, output), 0)
        output = self.o2o(output_combined)
        output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(self.hidden_size)

In [10]:
def train(local_batch, local_labels):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    loss = 0
    #print(local_batch[0])
    #print(local_batch.size())
    #print(local_labels[0])
    #print(local_labels.size())
    # parcourir le mini batc
    for i in range(len(local_batch)):
        hidden = rnn.initHidden()
        # parcourir la sequence
        for j in range(len(local_batch[i])):      
            output, hidden = rnn(local_batch[i][j], hidden)
            
        #print(output)
        #print(local_labels[i])
        #l = criterion(output, torch.tensor([torch.argmax(local_labels[i])]))
        #l = criterion(output, local_labels[i])
        l = criterion(torch.reshape(output,(1, len(output))), torch.tensor([torch.argmax(local_labels[i])]))
        loss += l

    loss.backward()

    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item() / len(local_batch)

In [12]:
rnn = RNN(len(dictChord), 128, len(dictChord))

plot_every = 1
all_losses = []
total_loss = 0 # Reset every plot_every iters
start = time.time()
criterion = nn.NLLLoss()
learning_rate = 0.0005
print_every = 1
max_epochs = 20


# Begin training

for epoch in range(1, max_epochs):
    # Training
    for local_batch, local_labels in training_generator:
        # Transfer to GPU
        local_batch, local_labels = local_batch.to(device), local_labels.to(device)
        
        output, loss = train(local_batch, local_labels)
        total_loss += loss

    if epoch % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), epoch, epoch / max_epochs * 100, loss))

    if epoch % plot_every == 0:
        all_losses.append(total_loss / (plot_every * len(local_batch)))
        total_loss = 0
        

    # Testing
    #for local_batch, local_labels in testing_generator:
        # Transfer to GPU
        #local_batch, local_labels = local_batch.to(device), local_labels.to(device)

        # //// Test the model  ////


RuntimeError: Expected a Tensor of type torch.cuda.FloatTensor but found a type torch.FloatTensor for sequence element 1 in sequence argument at position #1 'tensors'

In [ ]:
plt.figure()
plt.plot(all_losses)
plt.show()

In [56]:
test_sequence = ["C:maj","F:maj","C:maj","C:maj","F:maj","F:maj","C:maj","C:maj","G:maj","F:maj"]

test_sequence_tensor = torch.zeros(len(test_sequence), len(dictChord))
for t in range(len(test_sequence)):
    test_sequence_tensor[t, dictChord[test_sequence[t]]] = 1


In [57]:
hidden = rnn.initHidden()
generation_lenght = 120

generated_sequence = [0 for i in range(generation_lenght)]

for t in range(generation_lenght):
    if t < len(test_sequence):
        output, hidden = rnn(test_sequence_tensor[t], hidden)
        generated_sequence[t] = test_sequence[t]
    else : 
        last_chord_output = torch.zeros(len(dictChord))
        last_chord_output[torch.argmax(output).item()]
        output, hidden = rnn(last_chord_output, hidden)
        generated_sequence[t] = listChord[torch.argmax(output).item()]

        
for i in range(generation_lenght):
    if i%8 == 0:
        print(generated_sequence[i:i+8])

['C:maj', 'F:maj', 'C:maj', 'C:maj', 'F:maj', 'F:maj', 'C:maj', 'C:maj']
['G:maj', 'F:maj', 'F:maj', 'F:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj']
['C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj']
['C:maj', 'C:maj', 'C:maj', 'F:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj']
['F:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj']
['C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj']
['C:maj', 'C:maj', 'C:maj', 'C:maj', 'F:maj', 'C:maj', 'C:maj', 'C:maj']
['C:maj', 'C:maj', 'C:maj', 'C:maj', 'F:maj', 'C:maj', 'C:maj', 'C:maj']
['C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj']
['C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj']
['C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj', 'F:maj', 'C:maj']
['C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj', 'F:maj', 'C:maj', 'C:maj']
['C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:maj']
['C:maj', 'C:maj', 'C:maj', 'F:maj', 'C:maj', 'C:ma